In [19]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [20]:
import json
import httpx
import string
import secrets

from time import sleep

from getpass import getpass

from app.models import DeploymentContext
from app.auth import token_to_payload

<IPython.core.display.Javascript object>

In [21]:
base_url = "http://localhost:8000"
username = "jochen"
password = getpass()
service = "single_cast_hosting"

# get access_token
client = httpx.Client(base_url=base_url)
r = client.post("/token", data={"username": username, "password": password})
access_token = r.json()["access_token"]
access_token_headers = {"authorization": f"Bearer {access_token}"}

# get service token
data = {"service": service, "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers={"authorization": f"Bearer {service_token}"}
client.headers = headers

 ········


<IPython.core.display.Javascript object>

# Deploy

In [4]:
def show_details(details_url):
    r = client.get(details_url, headers=headers)
    rdata = r.json()
    len_steps = len([s for s in rdata["steps"] if s["state"] == "success"])
    started = rdata["started"]
    finished = rdata["finished"]
    print(f"{len_steps} {started} {finished}")

<IPython.core.display.Javascript object>

In [6]:
alphabet = string.ascii_letters + string.digits
database_password = ''.join(secrets.choice(alphabet) for i in range(20))
secret_key = "".join(secrets.choice(alphabet) for i in range(32))

<IPython.core.display.Javascript object>

## Create Service

### Get Service Token

In [5]:
data = {"service": service, "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers = {"authorization": f"Bearer {service_token}"}

<IPython.core.display.Javascript object>

In [13]:
alphabet = string.ascii_letters + string.digits
database_password = ''.join(secrets.choice(alphabet) for i in range(20))
secret_key = "".join(secrets.choice(alphabet) for i in range(32))
fqdn = "foobar.staging.django-cast.com"
underscored_fqdn = fqdn.replace(".", "_")
site_id = f"cast_{underscored_fqdn}"
env = {
    "fqdn": fqdn,
    "site_id": site_id,
    "user_name": "cast_1",
    "database_name": site_id,
    "database_user": site_id,
    "database_password": database_password,
    "secret_key": secret_key,
    "port": 10001,
    "settings_file_name": site_id
}
env

{'fqdn': 'foobar.staging.django-cast.com',
 'site_id': 'cast_foobar_staging_django-cast_com',
 'user_name': 'cast_1',
 'database_name': 'cast_foobar_staging_django-cast_com',
 'database_user': 'cast_foobar_staging_django-cast_com',
 'database_password': 'hgFCJVd45yWxxe3KhV6q',
 'secret_key': '7Hn1bYZAul3LUMI6MHfKIjTRtSIqVYep',
 'port': 10001,
 'settings_file_name': 'cast_foobar_staging_django-cast_com'}

<IPython.core.display.Javascript object>

In [17]:
context = DeploymentContext(env=env)
r = client.post("/deployments/", json=context.dict(), headers=headers)
rdata = r.json()
print(rdata)
details_url = r.json()["details"]

# for i in range(3):
#     show_details(details_url)
#     sleep(1)

{'id': 143, 'service_id': 6, 'origin': 'notebook', 'user': 'jochen', 'started': '2022-01-24T06:35:10.792402', 'finished': None, 'context': {'env': {'fqdn': 'foobar.staging.django-cast.com', 'site_id': 'cast_foobar_staging_django-cast_com', 'user_name': 'cast_1', 'database_name': 'cast_foobar_staging_django-cast_com', 'database_user': 'cast_foobar_staging_django-cast_com', 'database_password': 'hgFCJVd45yWxxe3KhV6q', 'secret_key': '7Hn1bYZAul3LUMI6MHfKIjTRtSIqVYep', 'port': 10001, 'settings_file_name': 'cast_foobar_staging_django-cast_com'}}, 'type': 'deployment', 'deleted': False, 'details': 'http://localhost:8000/deployments/143'}


<IPython.core.display.Javascript object>

## Remove Service

### Get Service Token

In [22]:
data = {"service": "remove_single_cast_hosting", "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers={"authorization": f"Bearer {service_token}"}

<IPython.core.display.Javascript object>

In [23]:
context = DeploymentContext(env=env)
r = client.post("/deployments/", json=context.dict(), headers=headers)
rdata = r.json()
print(rdata)
details_url = r.json()["details"]

{'id': 144, 'service_id': 7, 'origin': 'notebook', 'user': 'jochen', 'started': '2022-01-24T06:40:03.738147', 'finished': None, 'context': {'env': {'fqdn': 'foobar.staging.django-cast.com', 'site_id': 'cast_foobar_staging_django-cast_com', 'user_name': 'cast_1', 'database_name': 'cast_foobar_staging_django-cast_com', 'database_user': 'cast_foobar_staging_django-cast_com', 'database_password': 'hgFCJVd45yWxxe3KhV6q', 'secret_key': '7Hn1bYZAul3LUMI6MHfKIjTRtSIqVYep', 'port': 10001, 'settings_file_name': 'cast_foobar_staging_django-cast_com'}}, 'type': 'deployment', 'deleted': False, 'details': 'http://localhost:8000/deployments/144'}


<IPython.core.display.Javascript object>

In [35]:
context.env

{'fqdn': 'foobar.staging.wersdoerfer.de',
 'database_name': 'cast_foobar.staging.wersdoerfer.de',
 'database_user': 'cast_foobar.staging.wersdoerfer.de',
 'database_password': '6It0n1QGoMM4GtAkGmRr',
 'secret_key': '2oCTxJIo9YBpWquiVG5Ks3cJHcIPKZUB',
 'port': 10001,
 'settings_file_name': 'foobar_staging_wersdoerfer_de'}

<IPython.core.display.Javascript object>

# Stuff

In [31]:
foo = {"bar": "baz"}

<IPython.core.display.Javascript object>

In [32]:
dict(foo, blub="bla")

{'bar': 'baz', 'blub': 'bla'}

<IPython.core.display.Javascript object>

In [33]:
dict(foo, bar="blub")

{'bar': 'blub'}

<IPython.core.display.Javascript object>